<a href="https://colab.research.google.com/github/haydenhaotang/ColgateHP/blob/main/Rika_Recipe_Digital_Camp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#@title
!pip install snowflake | grep -v 'already satisfied'
import snowflake.connector
import sys
import re
con = snowflake.connector.connect(
    user="HAYDEN_TANG@COLPAL.COM", #You can get it by executing in UI: desc user <username>;
    account="colgatepalmolivedev.us-central1.gcp", #Add all of the account-name between https:// and snowflakecomputing.com
    authenticator="externalbrowser",#https://colgatepalmolivedev.us-central1.gcp.snowflakecomputing.com
)
!pip install pygsheets | grep -v 'already satisfied'
import scipy.stats as st
import google.auth
import pandas as pd
import numpy as np
from google.colab import auth
import warnings
warnings.filterwarnings("ignore")
auth.authenticate_user()
import pygsheets
credentials, _ = google.auth.default()
gc = pygsheets.client.Client(credentials)
googlesheetid= input("输入Googlesheets 的url:")

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://sso.colpal.com/app/snowflake/exk5d2wdao1LeZ8Vr357/sso/saml?SAMLRequest=pZPNctowFIVfxaOubckG8qMBMg6QxlMnuOAwnexUWxAVWXJ1ZUzevrIJnXSRbLrTSOfofvceaXxzrKR34AaEVhMUBgR5XBW6FGo3QU%2F5nX%2BFPLBMlUxqxSfolQO6mY6BVbKmcWNf1Ir%2FbjhYz12kgPYHE9QYRTUDAVSxigO1BV3HDymNAkJro60utETvLJ87GAA31hGeLSUIh%2FdibU0xbts2aAeBNjscEUIwucZO1Um%2BnPVH19MH%2BhCTYad3CifP3thuhTqN4DOsnycR0Ps8z%2Fxsuc6RF59RZ1pBU3Gz5uYgCv60Sk8A4Ahmy%2FRrnC%2ByOH1YpslmMV9sggb8gitrmAyDXVEHoHS7lWzPC13VjXV1ArfCW15iqXfCjSKZT1C9F2WWpgs%2BvBjKOSySoS0Gj%2Blqzb7Buvp%2Bfxtnxx96I6PZr8vRYV8gb3POOuqyTgAanqguYeu2SDTww9CPrnJC6GhIRyS4IOEz8uYuYaGY7Z3nNgC0Y5I1kz0aq2v8lxrz435URm3JdJjy56uNGYwuOwfu4kanF0T78mb6%2F3MZ4%2Ff3vT3PR5dYMs%2B0FMWrd6dNxezHgYZB2O%2BI0t%2F2UsorJmRcloYDuGCl1O3McGbdL7Cm4QhPT1X%2F%2FQfTPw



1.   二层bom如何拆是否需要拆，拆的话逻辑会复杂化，具体逻辑？配方卡Google sheets的 tab名能否固定？
2.   现有的形式是否可以满足需求，如果不能满足，希望从何处
进行优化。
3.   如可以接受该形式需要做的事：谁会使用该工具？将需要为使用该工具的同事申请snowflake
4.   如果不可接受该种形式，探索其他的解决方案。





`形式：如果有不一样的可以进行highlight，如果物料号不一致，则highlight物料号，如果数量不一致，则highlight 数量
关于二层bom：二层bom的位置都处于第一列，需要进行查询，
最后，能否把源代码去掉，只给一些输入权限。

In [4]:
# @title
def checkfloat(i):
  if type(i)==float:
    return True
  else:
    return False
def check_bom_correctness(w_id):
  original_table = gc.open_by_url(w_id)
  receipe_card=original_table.worksheet_by_title('CSS-FH LiYuan')
  raw_recipe=receipe_card.get_as_df(start='B10',include_tailing_empty=False)
  filter_receipe=raw_recipe[raw_recipe.iloc[:,1].apply(checkfloat)==True]
  filter_columns=['目标加入量 ', '最少加入量',  '最大加入量', '实际加入量']
  new_columns=['materialNo','target','Min',' Max','Actual']
  new_filter_columns=pd.concat([filter_receipe.iloc[:,0],filter_receipe[filter_columns]],axis=1)
  new_filter_columns.columns=new_columns
  receipe_total=pd.DataFrame(new_filter_columns.groupby('materialNo')['target'].sum().reset_index())
  batch_size=float(receipe_card.get_value('B8'))
  receipe_total['target']=receipe_total['target']*1000/batch_size
  receipe_num=receipe_card.get_value('B6').replace(' ','')
  cur = con.cursor()
  try:
    cur.execute('''USE WAREHOUSE HAYDEN_TANG''')
    cur.execute('''select BOM_COMPONENT,PLANT,BOM_USAGE,ALT_BOM_TEXT, QUANTITY from DEV_GTED_HUB.BOM.BOM__AS
WHERE Material='{}' and ALT_BOM_TEXT='200000070597/001/001'
LIMIT 1000
'''.format(receipe_num))
    bomdata=pd.DataFrame(cur.fetchall())
  finally:
    cur.close()
  cur.close()
  bomdata.columns=['BOM_COMPONENT','PLANT','BOM_USAGE','ALT_BOM_TEXT', 'QUANTITY']
  bom_total=pd.DataFrame(bomdata.groupby('BOM_COMPONENT')['QUANTITY'].sum().reset_index())
  bom_total.columns=['materialNo','target']
  print('Bom中有但是配方表中没有的货号如下：\n',set(bom_total['materialNo'])-set(receipe_total['materialNo']))
  print('Bom中没有但是配方表中有的货号如下：\n',set(receipe_total['materialNo'])-set(bom_total['materialNo']))
  common_list=set(receipe_total['materialNo'])&set(bom_total['materialNo'])
  t1=bom_total[bom_total['materialNo'].isin(common_list)]
  t2=receipe_total[receipe_total['materialNo'].isin(common_list)]
  t1=t1.round(3).set_index('materialNo')
  t2=t2.round(3).set_index('materialNo')
  print('BOM和配方卡中有差异的物料如下：')
  print(t1.compare(t2,result_names=("bom", "receipe")))

In [6]:
 original_table = gc.open_by_url(googlesheetid)

In [66]:
def find_second_layer():
  test_pattern=re.compile(r'.*[)]')
  if re.findall(re.compile(r'B[0-9]+',re.I),re.sub(test_pattern,'','预混罐开批时间：1)B05313950000',0).replace(' ',''))==[]:
    return False
  else:
    return True

['B05313950000']

In [80]:
cur = con.cursor()
for t_list in raw_recipe.iloc[:,0].apply(lambda x: re.findall(re.compile(r'B[0-9]+',re.I),re.sub(test_pattern,'',x,0).replace(' ',''))):
  if t_list==[]:
    pass
  else:
    new_code=t_list[0]
    print(new_code)
    cur.execute('''USE WAREHOUSE HAYDEN_TANG''')
    cur.execute('''select ALT_BOM_TEXT from DEV_GTED_HUB.BOM.BOM__AS
WHERE Material='{}' order by ALT_BOM_TEXT desc
LIMIT 1
'''.format(new_code))
    print(pd.DataFrame(cur.fetchall()))
    seconddata=pd.DataFrame(cur.fetchall())
    try:
      secondversion=seconddata.iloc[0,0]
    except:
      secondversion=False
    if secondversion:
      cur.execute('''USE WAREHOUSE HAYDEN_TANG''')
      cur.execute('''select BOM_COMPONENT,PLANT,BOM_USAGE,ALT_BOM_TEXT, QUANTITY from DEV_GTED_HUB.BOM.BOM__AS
WHERE Material='{}' and ALT_BOM_TEXT='{}'
LIMIT 1000
'''.format(new_code,secondversion))
      print(pd.DataFrame(cur.fetchall()))




B05313950000
                      0
0  200000068512/001/000


In [46]:
pattern=re.compile('LEE|liyuan|best',re.I)
print(re.findall(pattern,"this is test L"))
for sheet in original_table._sheet_list:
  sheet_title=sheet.title
  if len(re.findall(pattern,sheet_title))>0:
    receipe_card=original_table.worksheet_by_title(sheet_title)
    raw_recipe=receipe_card.get_as_df(start='A10',include_tailing_empty=False)
    # print(raw_recipe)
    filter_receipe=raw_recipe[raw_recipe.iloc[:,2].apply(checkfloat)==True]
    filter_columns=['目标加入量 ', '最少加入量',  '最大加入量', '实际加入量']
    new_columns=['materialNo','target','Min',' Max','Actual']
    new_filter_columns=pd.concat([filter_receipe.iloc[:,1],filter_receipe[filter_columns]],axis=1)
    new_filter_columns.columns=new_columns
    receipe_total=pd.DataFrame(new_filter_columns.groupby('materialNo')['target'].sum().reset_index())
    batch_size=float(receipe_card.get_value('B8'))
    receipe_total['target']=receipe_total['target']*1000/batch_size
    receipe_num=receipe_card.get_value('B6').replace(' ','')
    test_pattern=re.compile(r'.*[)]')
    re.findall(re.compile(r'B[0-9]+',re.I),re.sub(test_pattern,'','1=3)B05313950000',0).replace(' ',''))
    for t_list in raw_recipe.iloc[:,0].apply(lambda x: re.findall(re.compile(r'B[0-9]+',re.I),re.sub(test_pattern,'',x,0).replace(' ',''))):

    # second_layer=r
#     cur = con.cursor()
#     try:
#       cur.execute('''USE WAREHOUSE HAYDEN_TANG''')
#       cur.execute('''select BOM_COMPONENT,PLANT,BOM_USAGE,ALT_BOM_TEXT, QUANTITY from DEV_GTED_HUB.BOM.BOM__AS
# WHERE Material='{}' and ALT_BOM_TEXT='200000070597/001/001'
# LIMIT 1000
# '''.format(receipe_num))
#       bomdata=pd.DataFrame(cur.fetchall())
#     finally:
#       cur.close()
#     cur.close()
#     bomdata.columns=['BOM_COMPONENT','PLANT','BOM_USAGE','ALT_BOM_TEXT', 'QUANTITY']
#     bom_total=pd.DataFrame(bomdata.groupby('BOM_COMPONENT')['QUANTITY'].sum().reset_index())
#     bom_total.columns=['materialNo','target']
      # bom_total['target']=bom_total['target']/1000*batch_size
#     print('Bom中有但是配方表中没有的货号如下：\n',set(bom_total['materialNo'])-set(receipe_total['materialNo']))
#     print('Bom中没有但是配方表中有的货号如下：\n',set(receipe_total['materialNo'])-set(bom_total['materialNo']))
#     common_list=set(receipe_total['materialNo'])&set(bom_total['materialNo'])
#     t1=bom_total[bom_total['materialNo'].isin(common_list)]
#     t2=receipe_total[receipe_total['materialNo'].isin(common_list)]
#     t1=t1.round(3).set_index('materialNo')
#     t2=t2.round(3).set_index('materialNo')
#     print('BOM和配方卡中有差异的物料如下：')
#     print(t1.compare(t2,result_names=("bom", "receipe")))

[]
0           预混罐开批时间：
1     1)B05313950000
2           山梨醇-酶混合物
3                   
4          2-1) 山梨醇桶
           ...      
65              检测人：
66              编写者：
67                  
68                  
69                  
Name: , Length: 70, dtype: object


In [16]:
original_table.worksheets

<bound method Spreadsheet.worksheets of <Spreadsheet 'Copy of Formula-Colgate KCSS Enzyme Forest Herb Toothpaste（200L)' Sheets:2>>

In [10]:
dir(original_table)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_defaultFormat',
 '_fetch_sheets',
 '_id',
 '_jsonsheet',
 '_named_ranges',
 '_sheet_list',
 '_title',
 'add_worksheet',
 'client',
 'create_developer_metadata',
 'custom_request',
 'default_parse',
 'defaultformat',
 'del_worksheet',
 'delete',
 'export',
 'fetch_properties',
 'find',
 'get_developer_metadata',
 'id',
 'locale',
 'logger',
 'named_ranges',
 'permissions',
 'protected_ranges',
 'remove_permission',
 'replace',
 'share',
 'sheet1',
 'title',
 'to_json',
 'update_properties',
 'updated',
 'url',
 'worksheet',
 'worksheet_by_title',
 'worksheet_cls',
 'worksheets']

In [5]:
check_bom_correctness(googlesheetid)

Bom中有但是配方表中没有的货号如下：
 {'M24572A'}
Bom中没有但是配方表中有的货号如下：
 {'M24572', 'M42458', 'M42459'}
BOM和配方卡中有差异的物料如下：
               target            
                  bom     receipe
materialNo                       
B05313950000    3.750    3.733333
B0660318MCN   149.543  157.533333
B08915380000   13.500   13.533333
M03043         12.500   12.533333
M06448          2.200         2.2
M06526        407.150       409.8
M14264         48.857   48.866667
M21094          0.500    0.533333
